## Run Separately

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification

from torch.optim import AdamW

import torch
from torch import cuda
import torch.nn as nn
from torch.optim import SGD
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import seqeval

from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.scheme import IOB1


In [2]:
file_path = r'C:\Users\tarci\OneDrive\Área de Trabalho\ner_models\data\df_tokens_labeled_iob_bert_format.csv'
df_total = pd.read_csv(file_path, encoding='utf-8')
df_total.rename(columns = {'text':'sentence', 'iob_labels':'tags'}, inplace = True)

In [4]:
train_file_path = r'C:\Users\tarci\OneDrive\Área de Trabalho\ner_models\data\df_train_llms_tokens_labeled_iob_bert_format.csv'
test_file_path = r'C:\Users\tarci\OneDrive\Área de Trabalho\ner_models\data\df_test_llms_tokens_labeled_iob_bert_format.csv'

df_train = pd.read_csv(train_file_path, encoding='utf-8')
df_test = pd.read_csv(test_file_path, encoding='utf-8')

df_train.rename(columns = {'text':'sentence', 'iob_labels':'tags'}, inplace = True)
df_test.rename(columns = {'text':'sentence', 'iob_labels':'tags'}, inplace = True)

In [7]:
class BioBertNER(nn.Module):
  """
  Implement NN class based on distilbert pretrained from Hugging face.
  Inputs :
    tokens_dim : int specifyng the dimension of the classifier
  """

  def __init__(self, tokens_dim):
    super(BioBertNER,self).__init__()

    if type(tokens_dim) != int:
            raise TypeError('Please tokens_dim should be an integer')

    if tokens_dim <= 0:
          raise ValueError('Classification layer dimension should be at least 1')

    self.pretrained = BertForTokenClassification.from_pretrained("pucpr/biobertpt-all", num_labels = tokens_dim) #set the output of each token classifier = unique_lables


  def forward(self, input_ids, attention_mask, labels = None): #labels are needed in order to compute the loss
    """
  Forwad computation of the network
  Input:
    - inputs_ids : from model tokenizer
    - attention :  mask from model tokenizer
    - labels : if given the model is able to return the loss value
  """

    #inference time no labels
    if labels == None:
      out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask )
      return out

    out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask , labels = labels)
    return out

In [8]:
class NerDataset(torch.utils.data.Dataset):
  """
  Custom dataset implementation to get (text,labels) tuples
  Inputs:
   - df : dataframe with columns [tags, sentence]
  """

  def __init__(self, df):
    if not isinstance(df, pd.DataFrame):
      raise TypeError('Input should be a dataframe')

    if "tags" not in df.columns or "sentence" not in df.columns:
      raise ValueError("Dataframe should contain 'tags' and 'sentence' columns")

    tags_list = [i.split() for i in df["tags"].values.tolist()]
    texts = df["sentence"].values.tolist()

    self.texts = [tokenizer(text, padding = "max_length", max_length = 512, truncation = True, return_tensors = "pt") for text in texts]
    self.labels = [match_tokens_labels(text, tags) for text,tags in zip(self.texts, tags_list)]

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    batch_text = self.texts[idx]
    batch_labels = self.labels[idx]

    return batch_text, torch.LongTensor(batch_labels)

In [9]:
class MetricsTracking():
  """
  In order make the train loop lighter I define this class to track all the metrics that we are going to measure for our model.
  """
  def __init__(self):

    self.total_acc = 0
    self.total_f1 = 0
    self.total_precision = 0
    self.total_recall = 0

  def update(self, predictions, labels , ignore_token = -100):
    '''
    Call this function every time you need to update your metrics.
    Where in the train there was a -100, were additional token that we dont want to label, so remove them.
    If we flatten the batch its easier to access the indexed = -100
    '''
    predictions = predictions.flatten()
    labels = labels.flatten()

    predictions = predictions[labels != ignore_token]
    labels = labels[labels != ignore_token]

    predictions = predictions.to("cpu")
    labels = labels.to("cpu")

    acc = accuracy_score(labels,predictions)
    f1 = f1_score(labels, predictions, average = "macro")
    precision = precision_score(labels, predictions, average = "macro")
    recall = recall_score(labels, predictions, average = "macro")

    self.total_acc  += acc
    self.total_f1 += f1
    self.total_precision += precision
    self.total_recall  += recall

  def return_avg_metrics(self,data_loader_size):
    n = data_loader_size
    metrics = {
        "acc": round(self.total_acc / n ,3),
        "f1": round(self.total_f1 / n, 3),
        "precision" : round(self.total_precision / n, 3),
        "recall": round(self.total_recall / n, 3)
          }
    return metrics

In [10]:
def tags_2_labels(tags : str, tag2idx : dict):
  '''
  Method that takes a list of tags and a dictionary mapping and returns a list of labels (associated).
  Used to create the "label" column in df from the "tags" column.
  '''
  return [tag2idx[tag] if tag in tag2idx else unseen_label for tag in tags.split()]

In [11]:
def tags_mapping(tags_series : pd.Series):
  """
  tag_series = df column with tags for each sentence.
  Returns:
    - dictionary mapping tags to indexes (label)
    - dictionary mappign inedexes to tags
    - The label corresponding to tag 'O'
    - A set of unique tags ecountered in the trainind df, this will define the classifier dimension
  """

  if not isinstance(tags_series, pd.Series):
      raise TypeError('Input should be a padas Series')

  unique_tags = set()

  for tag_list in df_train["tags"]:
    for tag in tag_list.split():
      unique_tags.add(tag)


  tag2idx = {k:v for v,k in enumerate(sorted(unique_tags))}
  idx2tag = {k:v for v,k in tag2idx.items()}

  unseen_label = tag2idx["O"]

  return tag2idx, idx2tag, unseen_label, unique_tags

In [12]:
def match_tokens_labels(tokenized_input, tags, ignore_token = -100):
        '''
        Used in the custom dataset.
        -100 will be tha label used to match additional tokens like [CLS] [PAD] that we dont care about.
        Inputs :
          - tokenized_input : tokenizer over the imput text -> {input_ids, attention_mask}
          - tags : is a single label array -> [O O O O O O O O O O O O O O B-tim O]

        Returns a list of labels that match the tokenized text -> [-100, 3,5,6,-100,...]
        '''

        #gives an array [ None , 0 , 1 ,2 ,... None]. Each index tells the word of reference of the token
        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(ignore_token)

            #if its equal to the previous word we can add the same label id of the provious or -100
            else :
                try:
                  reference_tag = tags[word_idx]
                  label_ids.append(tag2idx[reference_tag])
                except:
                  label_ids.append(ignore_token)

            previous_word_idx = word_idx

        return label_ids

In [13]:
def get_labels_unique_word(tokenized_input, predictions):
    word_ids = tokenized_input.word_ids()
    
    previous_word_idx = -1
    unique_tags_pred = []

    for word_idx in word_ids:
        if word_idx is None or (word_idx == previous_word_idx):
            continue

        else:
            idx_in_word_ids_array = word_ids.index(word_idx)
            reference_tag = predictions[idx_in_word_ids_array]
            unique_tags_pred.append(idx2tag[reference_tag])

        previous_word_idx = word_idx
  
    return unique_tags_pred

In [14]:
def train_loop_wt_eval(model, train_dataset, optimizer,  batch_size, epochs):

  train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  for epoch in range(epochs) :

    train_metrics = MetricsTracking()
    total_loss_train = 0

    model.train() #train mode

    for train_data, train_label in tqdm(train_dataloader):
      train_label = train_label.to(device)
      '''
      squeeze in order to match the sizes. From [batch,1,seq_len] --> [batch,seq_len]
      '''
      mask = train_data['attention_mask'].squeeze(1).to(device)
      input_id = train_data['input_ids'].squeeze(1).to(device)

      optimizer.zero_grad()

      output = model(input_id, mask, train_label)
      loss, logits = output.loss, output.logits
      predictions = logits.argmax(dim= -1)

      #compute metrics
      train_metrics.update(predictions, train_label)
      total_loss_train += loss.item()

      #grad step
      loss.backward()
      optimizer.step()

    train_results = train_metrics.return_avg_metrics(len(train_dataloader))

    print(f"TRAIN \nLoss: {total_loss_train / len(train_dataset)} \nMetrics {train_results}\n" )

In [15]:
def train_loop(model, train_dataset, dev_dataset, optimizer,  batch_size, epochs):

  train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
  dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  for epoch in range(epochs) :

    train_metrics = MetricsTracking()
    total_loss_train = 0

    model.train() #train mode

    for train_data, train_label in tqdm(train_dataloader):
      train_label = train_label.to(device)
      '''
      squeeze in order to match the sizes. From [batch,1,seq_len] --> [batch,seq_len]
      '''
      mask = train_data['attention_mask'].squeeze(1).to(device)
      input_id = train_data['input_ids'].squeeze(1).to(device)

      optimizer.zero_grad()

      output = model(input_id, mask, train_label)
      loss, logits = output.loss, output.logits
      predictions = logits.argmax(dim= -1)

      #compute metrics
      train_metrics.update(predictions, train_label)
      total_loss_train += loss.item()

      #grad step
      loss.backward()
      optimizer.step()

    '''
    EVALUATION MODE
    '''
    model.eval()

    dev_metrics = MetricsTracking()
    total_loss_dev = 0

    with torch.no_grad():
      for dev_data, dev_label in dev_dataloader:

        dev_label = dev_label.to(device)

        mask = dev_data['attention_mask'].squeeze(1).to(device)
        input_id = dev_data['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask, dev_label)
        loss, logits = output.loss, output.logits

        
        predictions = logits.argmax(dim= -1)

        dev_metrics.update(predictions, dev_label)
        total_loss_dev += loss.item()

    train_results = train_metrics.return_avg_metrics(len(train_dataloader))
    dev_results = dev_metrics.return_avg_metrics(len(dev_dataloader))

    print(f"TRAIN \nLoss: {total_loss_train / len(train_dataset)} \nMetrics {train_results}\n" )
    print(f"VALIDATION \nLoss {total_loss_dev / len(dev_dataset)} \nMetrics{dev_results}\n" )

In [16]:
#create tag-label mapping
#tag2idx, idx2tag , unseen_label, unique_tags = tags_mapping(df_train["tags"])
tag2idx, idx2tag , unseen_label, unique_tags = tags_mapping(df_total["tags"])

#create the label column from tag. Unseen labels will be tagged as "O"
#for df in [df_train, df_dev, df_test]:
#  df["labels"] = df["tags"].apply(lambda tags : tags_2_labels(tags, tag2idx))

for df in [df_train, df_test]:
  df["labels"] = df["tags"].apply(lambda tags : tags_2_labels(tags, tag2idx))

In [17]:
#original text
#text = df_train["sentence"].values.tolist()
text = df_total["sentence"].values.tolist()

#toeknized text
tokenizer = AutoTokenizer.from_pretrained("pucpr/biobertpt-all", do_lower_case=False)
text_tokenized = tokenizer(text , padding = "max_length", max_length = 512, truncation = True, return_tensors = "pt" )

#mapping token to original word
word_ids = text_tokenized.word_ids()

In [18]:
#datasets
train_dataset = NerDataset(df_train)
#dev_dataset = NerDataset(df_dev)

## Model Test

In [ ]:
model_test = BioBertNER(len(unique_tags))
optimizer_test = Adam(model_test.parameters(), lr=learning_rate_hp[0])

parameters_test = {
    "model": model_test,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_test,
    "batch_size" : 4,
    "epochs" : 1
}

In [ ]:
train_loop_wt_eval(**parameters_test)

In [24]:
#dir_path = ''
#torch.save(model_test.state_dict(), dir_path)

In [ ]:
model_load = MultiBertNER(tokens_dim=len(unique_tags))
model_load.load_state_dict(torch.load(dir_path))

In [ ]:
parameters_test_load = {
    "model": model_load,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_test,
    "batch_size" : 4,
    "epochs" : 5
}

In [ ]:
train_loop_wt_eval(**parameters_test_load)

## Hyperparameters

In [19]:
# Hyperparam
learning_rate_hp = [2e-3, 1e-3]
epochs = [5, 10]
batch_size_hp = [4, 8, 16]

## Model 00

In [ ]:
model_00 = BioBertNER(len(unique_tags))

optimizer_00 = SGD(model_00.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_00 = {
    "model": model_00,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_00,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[0]
}

In [ ]:
train_loop_wt_eval(**parameters_00)

In [22]:
#dir_path = ''
torch.save(model_00.state_dict(), 'model_00.pth')


## Model 01

In [ ]:
model_01 = BioBertNER(len(unique_tags))
optimizer_01 = SGD(model_01.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_01 = {
    "model": model_01,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_01,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[1]
}

In [ ]:
train_loop_wt_eval(**parameters_01)

In [22]:
#dir_path = ''
torch.save(model_01.state_dict(), 'model_01.pth')

## Model 02

In [ ]:
model_02 = BioBertNER(len(unique_tags))
optimizer_02 = SGD(model_02.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_02 = {
    "model": model_02,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_02,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[2]
}

In [ ]:
train_loop_wt_eval(**parameters_02)

In [22]:
#dir_path = ''
torch.save(model_02.state_dict(), 'model_02.pth')

## Model 03

In [ ]:
model_03 = BioBertNER(len(unique_tags))
optimizer_03 = SGD(model_03.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_03 = {
    "model": model_03,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_03,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[0]
}

In [ ]:
train_loop_wt_eval(**parameters_03)

In [23]:
#dir_path = ''
torch.save(model_03.state_dict(), 'model_03.pth')

## Model 04

In [ ]:
model_04 = BioBertNER(len(unique_tags))
optimizer_04 = SGD(model_04.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_04 = {
    "model": model_04,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_04,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[1]
}

In [ ]:
train_loop_wt_eval(**parameters_04)

In [22]:
#dir_path = ''
torch.save(model_04.state_dict(), 'model_04.pth')

## Model 05

In [ ]:
model_05 = BioBertNER(len(unique_tags))
optimizer_05 = SGD(model_05.parameters(), lr=learning_rate_hp[0], momentum = 0.9)

parameters_05 = {
    "model": model_05,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_05,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[2]
}

In [ ]:
train_loop_wt_eval(**parameters_05)

In [22]:
#dir_path = ''
torch.save(model_05.state_dict(), 'model_05.pth')

## Model 06

In [ ]:
model_06 = BioBertNER(len(unique_tags))
optimizer_06 = SGD(model_06.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_06 = {
    "model": model_06,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_06,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[0]
}

In [ ]:
train_loop_wt_eval(**parameters_06)

In [22]:
#dir_path = ''
torch.save(model_06.state_dict(), 'model_06.pth')

## Model 07

In [ ]:
model_07 = BioBertNER(len(unique_tags))
optimizer_07 = SGD(model_07.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_07 = {
    "model": model_07,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_07,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[1]
}

In [ ]:
train_loop_wt_eval(**parameters_07)

In [22]:
#dir_path = ''
torch.save(model_07.state_dict(), 'model_07.pth')

## Model 08

In [ ]:
model_08 = BioBertNER(len(unique_tags))
optimizer_08 = SGD(model_08.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_08 = {
    "model": model_08,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_08,
    "epochs" : epochs[0],
    "batch_size" : batch_size_hp[2]
}

In [ ]:
train_loop_wt_eval(**parameters_08)

In [23]:
#dir_path = ''
torch.save(model_08.state_dict(), 'model_08.pth')

## Model 09

In [ ]:
model_09 = BioBertNER(len(unique_tags))
optimizer_09 = SGD(model_09.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_09 = {
    "model": model_09,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_09,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[0]
}

In [ ]:
train_loop_wt_eval(**parameters_09)

In [22]:
#dir_path = ''
torch.save(model_09.state_dict(), 'model_09.pth')

## Model 10

In [ ]:
model_10 = BioBertNER(len(unique_tags))
optimizer_10 = SGD(model_10.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_10 = {
    "model": model_10,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_10,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[1]
}

In [ ]:
train_loop_wt_eval(**parameters_10)

In [22]:
#dir_path = ''
torch.save(model_10.state_dict(), 'model_10.pth')

## Model 11

In [ ]:
model_11 = BioBertNER(len(unique_tags))
optimizer_11 = SGD(model_11.parameters(), lr=learning_rate_hp[1], momentum = 0.9)

parameters_11 = {
    "model": model_11,
    "train_dataset": train_dataset,
    "optimizer" : optimizer_11,
    "epochs" : epochs[1],
    "batch_size" : batch_size_hp[2]
}

In [ ]:
train_loop_wt_eval(**parameters_11)

In [22]:
#dir_path = ''
torch.save(model_11.state_dict(), 'model_11.pth')

# Evaluate

In [19]:
def retrieve_token_tag_and_tag_pre(text, text_tokenized, tag, predictions, dev_label, tokenizer):
    word_ids = text_tokenized.word_ids()
    previous_index = None

    retrieved_tags_pred = []
    retrieved_tags_dev = []
    i = 0
    predictions = predictions[0]
    dev_label = dev_label[0]

    input_id = text_tokenized['input_ids']

    for word_idx in word_ids:
        
        if word_idx == None:
            pass
        elif word_idx == previous_index:
            pass
        else:
            retrieved_tags_pred.append(idx2tag[predictions[i]])
            if dev_label[i] == -100 or dev_label[i] == "-100":
                retrieved_tags_dev.append("O")
            else:
                retrieved_tags_dev.append(idx2tag[dev_label[i]])

        i += 1
        previous_index = word_idx

    return retrieved_tags_dev, retrieved_tags_pred

In [20]:
def evaluate_test_texts(model, df_test, batch_size = 1):

    dev_dataset = NerDataset(df_test)
    dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    dev_metrics = MetricsTracking()
    total_loss_dev = 0

    i = 0

    text_labels_dev = []
    text_labels_pred = []
    with torch.no_grad():
        for dev_data, dev_label in dev_dataloader:
            dev_label = dev_label.to(device)

            mask = dev_data['attention_mask'].squeeze(1).to(device)
            input_id = dev_data['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask, dev_label)
            loss, logits = output.loss, output.logits
        
            predictions = logits.argmax(dim= -1)

            tag = df_test.tags.iloc[i]
            text = df_test.sentence.iloc[i]

            text_tokenized = tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            labels_dev, labels_pred = retrieve_token_tag_and_tag_pre(text, text_tokenized, tag, predictions.tolist(), dev_label.tolist(), tokenizer)
            text_labels_dev.append(labels_dev)
            text_labels_pred.append(labels_pred)

            dev_metrics.update(predictions, dev_label)
            total_loss_dev += loss.item()
            i += 1
            

    dev_results = dev_metrics.return_avg_metrics(len(dev_dataloader))

    print(f"VALIDATION \nLoss {total_loss_dev / len(dev_dataset)} \nMetrics{dev_results}\n" )

    return text_labels_dev, text_labels_pred

# Carregar Modelos

## Model 00

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_00.pth"
model_00 = BioBertNER(tokens_dim=len(unique_tags))
model_00.load_state_dict(torch.load(dir_path))

In [ ]:
dev_00, pred_00 = evaluate_test_texts(model_00, df_test)

In [23]:
result_dict_00 = classification_report(dev_00, pred_00, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_00)

## Model 01

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_01.pth"
model_01 = BioBertNER(tokens_dim=len(unique_tags))
model_01.load_state_dict(torch.load(dir_path))

In [ ]:
dev_01, pred_01 = evaluate_test_texts(model_01, df_test)

In [27]:
result_dict_01 = classification_report(dev_01, pred_01, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_01)

## Model 02

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_02.pth"
model_02 = BioBertNER(tokens_dim=len(unique_tags))
model_02.load_state_dict(torch.load(dir_path))

In [ ]:
dev_02, pred_02 = evaluate_test_texts(model_02, df_test)

In [31]:
result_dict_02 = classification_report(dev_02, pred_02, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_02)

## Model 03

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_03.pth"
model_03 = BioBertNER(tokens_dim=len(unique_tags))
model_03.load_state_dict(torch.load(dir_path))

In [ ]:
dev_03, pred_03 = evaluate_test_texts(model_03, df_test)

In [35]:
result_dict_03 = classification_report(dev_03, pred_03, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_03)

## Model 04

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_04.pth"
model_04 = BioBertNER(tokens_dim=len(unique_tags))
model_04.load_state_dict(torch.load(dir_path))

In [ ]:
dev_04, pred_04 = evaluate_test_texts(model_04, df_test)

In [39]:
result_dict_04 = classification_report(dev_04, pred_04, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_04)

## Model 05

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_05.pth"
model_05 = BioBertNER(tokens_dim=len(unique_tags))
model_05.load_state_dict(torch.load(dir_path))

In [ ]:
dev_05, pred_05 = evaluate_test_texts(model_05, df_test)

In [43]:
result_dict_05 = classification_report(dev_05, pred_05, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_05)

## Model 06

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_06.pth"
model_06 = BioBertNER(tokens_dim=len(unique_tags))
model_06.load_state_dict(torch.load(dir_path))

In [ ]:
dev_06, pred_06 = evaluate_test_texts(model_06, df_test)

In [47]:
result_dict_06 = classification_report(dev_06, pred_06, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_06)

## Model 07

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_07.pth"
model_07 = BioBertNER(tokens_dim=len(unique_tags))
model_07.load_state_dict(torch.load(dir_path))

In [ ]:
dev_07, pred_07 = evaluate_test_texts(model_07, df_test)

In [51]:
result_dict_07 = classification_report(dev_07, pred_07, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_07)

## Model 08

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_08.pth"
model_08 = BioBertNER(tokens_dim=len(unique_tags))
model_08.load_state_dict(torch.load(dir_path))

In [ ]:
dev_08, pred_08 = evaluate_test_texts(model_08, df_test)

In [55]:
result_dict_08 = classification_report(dev_08, pred_08, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_08)

## Model 09

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_09.pth"
model_09 = BioBertNER(tokens_dim=len(unique_tags))
model_09.load_state_dict(torch.load(dir_path))

In [ ]:
dev_09, pred_09 = evaluate_test_texts(model_09, df_test)

In [59]:
result_dict_09 = classification_report(dev_09, pred_09, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_09)

## Model 10

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_10.pth"
model_10 = BioBertNER(tokens_dim=len(unique_tags))
model_10.load_state_dict(torch.load(dir_path))

In [ ]:
dev_10, pred_10 = evaluate_test_texts(model_10, df_test)

In [63]:
result_dict_10 = classification_report(dev_10, pred_10, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_10)

## Model 11

In [ ]:
dir_path = r"C:\Users\tarci\OneDrive\Área de Trabalho\biobertpt\model_11.pth"
model_11 = BioBertNER(tokens_dim=len(unique_tags))
model_11.load_state_dict(torch.load(dir_path))

In [ ]:
dev_11, pred_11 = evaluate_test_texts(model_11, df_test)

In [67]:
result_dict_11 = classification_report(dev_11, pred_11, mode="strict", scheme=IOB2, zero_division=False)

In [ ]:
print(result_dict_11)